### import necessary pakages

In [79]:
import ee
import geemap
import numpy as np
import rasterio
from rasterio.transform import from_origin
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import shape, mapping
import json
import os

### Earth Engine Authentication

In [80]:
# ee.Authenticate()
ee.Initialize(project='encoded-blend-391808')

### Load ROI

In [81]:
# cell 2
roi_path = r"..\input\roi.geojson"
with open(roi_path) as f:
    gj = json.load(f)

feature = gj['features'][0]
geom = feature['geometry']
ee_roi = ee.Geometry(feature['geometry'])
print("ROI loaded:", feature.get('properties', {}).get('name', 'ROI'))

ROI loaded: bengaluru


### get sentinel-2 data

In [82]:
# cell 3
def get_s2_composite(roi, start_date, end_date):
    s2 = (ee.ImageCollection('COPERNICUS/S2_SR')
            .filterBounds(roi)
            .filterDate(start_date, end_date)
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60))
         )

    # --- SCL-based cloud/shadow/snow mask ---
    def mask_s2_scl(img):
        scl = img.select('SCL')

        # Allowed SCL classes (keep): vegetation, bare soil, water, unclassified
        valid = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(7))

        # Remove clouds, cloud shadows, saturated, snow, cirrus
        return img.updateMask(valid)

    s2_masked = s2.map(mask_s2_scl)

    # Build median composite
    comp = s2_masked.median().clip(roi)

    # Commonly used S2 bands for indices
    bands = ['B2','B3','B4','B8','B11','B12']  # blue, green, red, nir, swir1, swir2
    return comp.select(bands)
pre_s2  = get_s2_composite(ee_roi, '2023-12-01', '2024-01-31')
post_s2 = get_s2_composite(ee_roi, '2024-12-01', '2025-01-31')

### get sentinel-1 data

In [83]:
# cell 4
def get_s1_composite(roi, start_date, end_date, polarization='VV', orbit='DESCENDING'):
    s1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
           .filterBounds(roi)
           .filterDate(start_date, end_date)
           .filter(ee.Filter.eq('instrumentMode', 'IW'))
           .filter(ee.Filter.eq('orbitProperties_pass', orbit))
           .filter(ee.Filter.eq('resolution_meters', 10))
         )
    # select polarization and take median
    comp = s1.select([polarization]).median().clip(roi)
    return comp

pre_s1 = get_s1_composite(ee_roi, '2023-12-01', '2024-01-31', 'VV', orbit='DESCENDING')
post_s1 = get_s1_composite(ee_roi, '2024-12-01', '2025-01-31', 'VV', orbit='DESCENDING')


#### save the images in local

In [ ]:
# cell 5
out_dir = r"..\output\image_input"
os.makedirs(out_dir, exist_ok=True)

def ee_image_to_local(img, filename, region, scale=10, crs='EPSG:3857'):
    path = os.path.join(out_dir, filename)
    geemap.ee_export_image(
        img,
        filename=path,
        scale=scale,
        crs=crs,
        region=region,
        file_per_band=False
    )
    print("Export queued:", path)
    return path

# ---- Export RGB True Color ----
rgb_pre  = pre_s2.select(['B4','B3','B2']).visualize(min=0, max=3000)
rgb_post = post_s2.select(['B4','B3','B2']).visualize(min=0, max=3000)

ee_image_to_local(rgb_pre,  "pre_s2_rgb.tif", ee_roi)
ee_image_to_local(rgb_post, "post_s2_rgb.tif", ee_roi)

# ---- Export raw for processing ----
ee_image_to_local(pre_s2,  "pre_s2_raw.tif", ee_roi)
ee_image_to_local(post_s2, "post_s2_raw.tif", ee_roi)

ee_image_to_local(pre_s1,  "pre_s1_vv.tif", ee_roi)
ee_image_to_local(post_s1, "post_s1_vv.tif", ee_roi)


### reading the necessary images

In [86]:
# cell 6
import rasterio
def read_raster(path):
    with rasterio.open(path) as src:
        arr = src.read().astype(np.float32)
        meta = src.meta.copy()
    return arr, meta

pre_s2_arr, pre_s2_meta = read_raster('../output/image_input/pre_s2_raw.tif')
post_s2_arr, post_s2_meta = read_raster('../output/image_input/post_s2_raw.tif')
pre_s1_arr, pre_s1_meta = read_raster('../output/image_input/pre_s1_vv.tif')
post_s1_arr, post_s1_meta = read_raster('../output/image_input/post_s1_vv.tif')

print("S2 shape (bands, h, w):", pre_s2_arr.shape)
print("S1 shape (bands, h, w):", pre_s1_arr.shape)


S2 shape (bands, h, w): (6, 175, 139)
S1 shape (bands, h, w): (1, 175, 139)


#### Converting dB to linear and applying the Lee filter to remove noise


In [87]:
from scipy.ndimage import uniform_filter
import numpy as np

def lee_filter(img, size=3):
    img_mean = uniform_filter(img, size=size)
    img_mean_sq = uniform_filter(img * img, size=size)
    img_var = img_mean_sq - img_mean * img_mean

    # noise variance
    noise_var = np.mean(img_var)

    # weight (local coefficient of variation)
    cu = img_var / (img_var + noise_var)
    return img_mean + cu * (img - img_mean)

# Use only band 0
pre_s1_int = pre_s1_arr[0]
post_s1_int = post_s1_arr[0]

print("pre_s1 min/max:", np.nanmin(pre_s1_int), np.nanmax(pre_s1_int))

# Detect dB by presence of negative values
if np.nanmin(pre_s1_int) < 0:
    print("Detected dB scale → converting to linear")
    pre_s1_lin = 10 ** (pre_s1_int / 10.0)
    post_s1_lin = 10 ** (post_s1_int / 10.0)
else:
    print("Detected linear scale → no conversion needed")
    pre_s1_lin = pre_s1_int
    post_s1_lin = post_s1_int

# Apply Lee filter
pre_s1_filt = lee_filter(pre_s1_lin, size=3)
post_s1_filt = lee_filter(post_s1_lin, size=3)

pre_s1 min/max: -21.849335 24.000546
Detected dB scale → converting to linear


#### Computing NDBI and applying scale


In [88]:
# cell 8
# pre_s2_arr shape: (bands, h, w) where bands follow the chosen order: B2,B3,B4,B8,B11,B12
B2,B3,B4,B8,B11,B12 = range(6)

def compute_ndbi(arr):
    # arr: bands x h x w
    swir = arr[B11,:,:]
    nir = arr[B8,:,:]
    # avoid zero division
    denom = (swir + nir).copy()
    denom[denom==0] = 1e-6
    ndbi = (swir - nir) / denom
    return ndbi

pre_ndbi = compute_ndbi(pre_s2_arr)
post_ndbi = compute_ndbi(post_s2_arr)
ndbi_diff = post_ndbi - pre_ndbi


### applying threshold for urbun area

In [89]:
optical_change_mask = (ndbi_diff > 0.05).astype(np.uint8)


### Converting linear back to dB scale and applying a threshold for urban areas


In [90]:
# cell 9
eps = 1e-6
log_pre = 10 * np.log10(pre_s1_filt + eps)
log_post = 10 * np.log10(post_s1_filt + eps)
s1_diff_db = log_post - log_pre

# threshold => e.g., abs(db) > 2.0 dB considered change (tuneable)
s1_change_mask = (np.abs(s1_diff_db) > 2.0).astype(np.uint8)


#### Converting both NDBI and log_ratio to the same scale, fusing them, and applying a threshold


In [91]:
# cell 10
# normalize ndbi_diff to 0-1 by clipping a reasonable range
ndbi_clip = np.clip(ndbi_diff, -0.2, 0.2)
ndbi_conf = (ndbi_clip - (-0.2)) / (0.4)  # map [-0.2,0.2] -> [0,1], positive part indicates built-up increase
ndbi_conf = np.clip(ndbi_conf, 0, 1)

# SAR confidence: abs(db diff) clipped to 0..6 dB
sar_clip = np.clip(np.abs(s1_diff_db), 0, 6)
sar_conf = sar_clip / 6.0

# fused confidence: weighted average (you can tune weights)
w_opt = 0.6
w_sar = 0.4
fused_conf = w_opt * ndbi_conf + w_sar * sar_conf

# fused binary mask threshold
fused_mask = (fused_conf > 0.6).astype(np.uint8)


### saving all the computed output

In [92]:
# cell 11
def save_png(arr, filename, cmap='gray'):
    plt.figure(figsize=(6,6))
    plt.imshow(arr, cmap=cmap)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(filename, dpi=150, bbox_inches='tight')
    plt.close()
    print("Saved:", filename)

# create an RGB for display from S2 arrays (scale reflectance appropriately)
def make_rgb_from_s2(arr):
    # arr band order B2,B3,B4... scale to 0..1 using a heuristic
    r = arr[2,:,:]
    g = arr[1,:,:]
    b = arr[0,:,:]
    def stretch(x):
        x = np.clip(x, 0, 3000)
        return (x / 3000.0)
    rgb = np.dstack([stretch(r), stretch(g), stretch(b)])
    return rgb

pre_rgb = make_rgb_from_s2(pre_s2_arr)
post_rgb = make_rgb_from_s2(post_s2_arr)

save_png(pre_rgb, "../output/pre_rgb.png")
save_png(post_rgb, "../output/post_rgb.png")
save_png(ndbi_diff, "../output/ndbi_diff.png", cmap='bwr')
save_png(s1_diff_db, "../output/s1_diff_db.png", cmap='bwr')
save_png(optical_change_mask, "../output/optical_mask.png", cmap='gray')
save_png(s1_change_mask, "../output/s1_mask.png", cmap='gray')
save_png(fused_conf, "../output/fused_conf.png", cmap='viridis')
save_png(fused_mask, "../output/fused_mask.png", cmap='gray')


Saved: ../output/pre_rgb.png
Saved: ../output/post_rgb.png
Saved: ../output/ndbi_diff.png
Saved: ../output/s1_diff_db.png
Saved: ../output/optical_mask.png
Saved: ../output/s1_mask.png
Saved: ../output/fused_conf.png
Saved: ../output/fused_mask.png


### saving the fused mask

In [93]:
def save_geotiff(array, meta, outpath, dtype=rasterio.uint8):
    # Ensure output folder exists
    os.makedirs(os.path.dirname(outpath), exist_ok=True)

    meta_copy = meta.copy()
    meta_copy.update({
        "dtype": dtype,
        "count": 1
    })

    with rasterio.open(outpath, 'w', **meta_copy) as dst:
        dst.write(array, 1)

# Now save file
os.makedirs("output", exist_ok=True)
save_geotiff(fused_mask, pre_s2_meta, "../output/fused_mask.tif", dtype=rasterio.uint8)

save_geotiff((fused_conf * 255).astype(np.uint8),
             pre_s2_meta,
             "../output/fused_confidence.tif",
             dtype=rasterio.uint8)


### saving others mask

In [95]:
# cell 14
union = ((optical_change_mask + s1_change_mask) > 0).astype(np.uint8)
intersect = ((optical_change_mask + s1_change_mask) == 2).astype(np.uint8)

save_png(union, "../output/union_mask.png", cmap='gray')
save_png(intersect, "../output/intersect_mask.png", cmap='gray')


Saved: ../output/union_mask.png
Saved: ../output/intersect_mask.png


### total changed area calculation and plot

In [102]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os

# ---------------------------
# 1. Read fused_mask GeoTIFF
# ---------------------------
tif_path = "../output/fused_mask.tif"

with rasterio.open(tif_path) as src:
    fused_mask = src.read(1)               # read first band
    transform = src.transform              # pixel transform
    crs = src.crs                          # projection
    width = src.width
    height = src.height

print("Projection:", crs)
print("Image size:", width, "x", height)


# ---------------------------
# 2. Calculate area (sq km)
# ---------------------------
# pixel size from affine transform (EPSG:3857 → meters)
pixel_width = abs(transform[0])
pixel_height = abs(transform[4])

pixel_area_m2 = pixel_width * pixel_height  # m²
pixel_area_km2 = pixel_area_m2 / 1_000_000  # km²

# total area
total_pixels = fused_mask.size
total_area_km2 = total_pixels * pixel_area_km2

# changed area (class == 1)
changed_pixels = np.sum(fused_mask == 1)
changed_area_km2 = changed_pixels * pixel_area_km2

print("Total area (sq km):", total_area_km2)
print("Changed area (sq km):", changed_area_km2)


# ---------------------------
# 3. Create Bar Graph (sq km)
# ---------------------------
labels = ['Total Area', 'Changed Area']
areas = [total_area_km2, changed_area_km2]

plt.figure(figsize=(8, 5))
bars = plt.bar(labels, areas)

# Add area labels
for bar, area in zip(bars, areas):
    plt.text(bar.get_x() + bar.get_width() / 2,
             bar.get_height(),
             f"{area:,.3f} km²",
             ha='center', va='bottom', fontsize=12)

plt.ylabel("Area (sq km)")
plt.title("Total Area vs Changed Area (Fused Mask)")

# save output
os.makedirs("output", exist_ok=True)
plt.savefig("output/area_bar_graph.png", dpi=300, bbox_inches="tight")
plt.close()

print("Saved bar graph: output/area_bar_graph.png")


Projection: EPSG:3857
Image size: 139 x 175
Total area (sq km): 2.4325
Changed area (sq km): 0.1862
Saved bar graph: output/area_bar_graph.png
